<a href="https://colab.research.google.com/github/kuzminadya/FHP-infovis-py/blob/master/Data_Dossier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Dossier | Consultation on 11 June

## An idea

My idea is to analyse and visualise the transformation of font file changing over time. Creating a font, especially a large font family may take several years of work or it might be a result of collaboration of several designers and type engineers. Even after the first version is released, the contribution to the project and updating the font may last for several years.

*add text about version control system*

## Title (the first version)

Visualisation and analysis of font file transformation over time. 

## Note

For now the notebook covers three different approaches. In the final version, I hope I'll choose and visualize only one. 

## An approximate work plan

[x] Choose a topic and/or dataset you are interested in\
[x] Prepare the data for the first use case\
[x] Prepare the data for the second use case\
[ ] Pose a question to apply your visualization skills → 11.06.2020\
[ ] Generate several visualizations (static and interactive) → 25.06.2020\
[ ] Include the most informative in report on your process → 02.07.2020\
[ ] Share the resulting report as a Jupyter Notebook → 02.02.2020


## Use cases

There are two possible ways of elaborating my experiment. I can visualise either the structure of the existing font (the first use case) or I can create one, using UFO format (the second use case). For the second story I need to prepare a sample dataset myself.

### The first use case

As an examle I use Source Code Pro Project from Adobe, which took part for 4 years:\
https://github.com/adobe-fonts/source-code-pro/commits/release

And IBM Plex® typeface which was active on Github for a 2 years and have 219 commits:\
https://github.com/IBM/plex 

### The second use case

For the second project I decided to use my own current project [Mogee font](http://nadyakuzmina.com/story-of-mogeefont)

The font originally was made in glyphs and I need to prepare all the existing versions in UFO format.

https://robofont.com/documentation/how-tos/using-git/

> The UFO file format is perfect for version control, since it is text-based. A UFO is basically just a folder with many little XML-files in it. A UFO file could theoretically be created and modified in a text editor. This opens the field of digital type design for version control tools normally used only by software developers.

## IBM Plex®

Question

https://github.com/IBM/plex/blob/master/LICENSE.txt


In [0]:
import pandas as pd
import altair as alt

The first step is getting the dataset in JSON format from Github. It includes two files: a description of the repo and a list of commits.

https://developer.github.com/v3/repos/#get-a-repository

/repos/:owner/:repo

In [3]:
import requests 

url = ('https://api.github.com/repos/IBM/plex')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_repo_json = response.json()

# dataset_repo_json

Request returned 200 : 'OK'


https://developer.github.com/v3/repos/commits/ 

GET /repos/:owner/:repo/commits

In [4]:
url = ('https://api.github.com/repos/IBM/plex/commits')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_commits_json = response.json()

# dataset_commits_json

Request returned 200 : 'OK'


The next part is converting JSON into table.

https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [5]:
from pandas.io.json import json_normalize
df_commits_table = pd.DataFrame.from_dict(json_normalize(dataset_commits_json), orient='columns')
df_repo_table = pd.DataFrame.from_dict(json_normalize(dataset_repo_json), orient='columns')
# df_commits_table
# df_repo_table

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


Then hiding unnecessary columns

https://colab.research.google.com/drive/11zDWTJHup4A4rEUZq2evGVpXagElfTdT?usp=sharing

In [6]:
df_commits_table_reduced = df_commits_table.drop(columns=["sha", "node_id", "parents", "commit.author.email", "commit.committer.email", "commit.tree.sha", "commit.verification.payload",  "author.login", "author.id", "author.node_id", "author.gravatar_id", "committer.id", "committer.node_id", "committer.gravatar_id"])

df_commits_table_reduced.sample(5)

,url,html_url,comments_url,commit.author.name,commit.author.date,commit.committer.name,commit.committer.date,commit.message,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,author.avatar_url,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.avatar_url,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,author,committer
18,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/2224b57a801...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-10-03T18:45:57Z,Vince Picone,2019-10-03T18:45:57Z,breaking: update thai and devanagari typeface ...,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/ef0ab499832...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-12-09T21:49:34Z,Vince Picone,2019-12-09T21:49:34Z,4.0.2,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/e805eb4106e...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-11-18T16:15:37Z,Vince Picone,2019-11-18T16:15:37Z,4.0.1,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/ceee77eadc0...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-04-05T18:34:31Z,Vince Picone,2019-04-05T18:34:31Z,chore: npm ignore .DS_Store,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/be666280d04...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-06-04T19:07:41Z,GitHub,2019-06-04T19:07:41Z,Update CHANGELOG.md,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,https://avatars1.githubusercontent.com/u/40780...,https://api.github.com/users/vpicone,https://github.com/vpicone,https://api.github.com/users/vpicone/followers,https://api.github.com/users/vpicone/following...,https://api.github.com/users/vpicone/gists{/gi...,https://api.github.com/users/vpicone/starred{/...,https://api.github.com/users/vpicone/subscript...,https://api.github.com/users/vpicone/orgs,https://api.github.com/users/vpicone/repos,https://api.github.com/users/vpicone/events{/p...,https://api.github.com/users/vpicone/received_...,User,False,web-flow,https://avatars3.githubusercontent.com/u/19864...,https://api.github.com/users/web-flow,https://github.com/web-flow,https://api.github.com/users/web-flow/followers,https://api.github.com/users/web-flow/followin...,htt

In [7]:
df_repo_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 115 entries, id to organization.site_admin
dtypes: bool(11), int64(13), object(91)
memory usage: 971.0+ bytes


In [8]:
df_commits_table_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 45 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            30 non-null     object 
 1   html_url                       30 non-null     object 
 2   comments_url                   30 non-null     object 
 3   commit.author.name             30 non-null     object 
 4   commit.author.date             30 non-null     object 
 5   commit.committer.name          30 non-null     object 
 6   commit.committer.date          30 non-null     object 
 7   commit.message                 30 non-null     object 
 8   commit.tree.url                30 non-null     object 
 9   commit.url                     30 non-null     object 
 10  commit.comment_count           30 non-null     int64  
 11  commit.verification.verified   30 non-null     bool   
 12  commit.verification.reason     30 non-null     objec

Then parsing *commit.committer.date          30 non-null     object* in datetime format


In [9]:
df_commits_table_reduced['commit.author.date'] = pd.to_datetime(df_commits_table_reduced['commit.author.date'], format='%Y-%m-%d')

df_commits_table_reduced['commit.author.date'].sample(5)

18   2019-10-03 18:45:57+00:00
11   2019-11-18 15:57:32+00:00
24   2019-04-08 00:27:52+00:00
7    2019-11-18 16:29:14+00:00
2    2019-12-09 21:48:01+00:00
Name: commit.author.date, dtype: datetime64[ns, UTC]

Couinting amount of commits for each day 

In [13]:
df_time_index = df_commits_table_reduced.set_index("commit.author.date")
commits_sums = df_time_index.resample("D").sum()
commits_sums

# ha-ha, doesn't work

,commit.comment_count,commit.verification.verified,author,committer
commit.author.date,,,,
2019-04-05 00:00:00+00:00,0,1.0,0.0,0.0
2019-04-06 00:00:00+00:00,0,0.0,0.0,0.0
2019-04-07 00:00:00+00:00,0,0.0,0.0,0.0
2019-04-08 00:00:00+00:00,0,1.0,0.0,0.0
2019-04-09 00:00:00+00:00,0,0.0,0.0,0.0
...,...,...,...,...
2020-05-08 00:00:00+00:00,0,0.0,0.0,0.0
2020-05-09 00:00:00+00:00,0,0.0,0.0,0.0
2020-05-10 00:00:00+00:00,0,0.0,0.0,0.0


A test visualisation:

it could be a visualisation of amount of commits each day + commit comments on hover

## Second use case

The second use case — analyse of a UFO font file

https://github.com/TiroTypeworks/Slabo for manipulation with UFOs



In [0]:
import xml.etree.ElementTree as ET

Decsribing of A glyph from Slabo 13px b004.ufo

```
<?xml version="1.0" encoding="UTF-8"?>
<glyph name="A" format="1">
  <advance width="480"/>
  <unicode hex="0041"/>
  <outline>
    <contour>
      <point x="120" y="55" type="line"/>
      <point x="120" y="0" type="line"/>
      <point x="-60" y="0" type="line"/>
      <point x="-60" y="55" type="line"/>
      <point x="-5" y="55" type="line"/>
      <point x="179" y="540" type="line"/>
      <point x="246" y="540" type="line"/>
      <point x="426" y="55" type="line"/>
      <point x="480" y="55" type="line"/>
      <point x="480" y="0" type="line"/>
      <point x="300" y="0" type="line"/>
      <point x="300" y="55" type="line"/>
      <point x="362" y="55" type="line"/>
      <point x="314" y="185" type="line"/>
      <point x="102" y="185" type="line"/>
      <point x="55" y="55" type="line"/>
    </contour>
    <contour>
      <point x="210" y="470" type="line"/>
      <point x="206" y="470" type="line"/>
      <point x="122" y="240" type="line"/>
      <point x="294" y="240" type="line"/>
    </contour>
  </outline>
</glyph>
```



In [0]:
glyph = []
advance_width = []
unicode = []
outline = []



Kerning Slabo 13px b004.ufo

```
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>F</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>a</key>
		<integer>-60</integer>
		<key>ampersand</key>
		<integer>-60</integer>
		<key>comma</key>
		<integer>-120</integer>
		<key>o</key>
		<integer>-60</integer>
		<key>s</key>
		<integer>-60</integer>
		<key>x</key>
		<integer>-60</integer>
	</dict>
	<key>L</key>
	<dict>
		<key>T</key>
		<integer>-60</integer>
		<key>V</key>
		<integer>-60</integer>
		<key>W</key>
		<integer>-60</integer>
		<key>Y</key>
		<integer>-60</integer>
	</dict>
	<key>P</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>comma</key>
		<integer>-60</integer>
	</dict>
	<key>T</key>
	<dict>
		<key>comma</key>
		<integer>-60</integer>
		<key>o</key>
		<integer>-60</integer>
	</dict>
	<key>V</key>
	<dict>
		<key>comma</key>
		<integer>-60</integer>
		<key>ibreve</key>
		<integer>60</integer>
		<key>icircumflex</key>
		<integer>60</integer>
		<key>idieresis</key>
		<integer>60</integer>
		<key>imacron</key>
		<integer>60</integer>
		<key>itilde</key>
		<integer>60</integer>
		<key>jcircumflex</key>
		<integer>60</integer>
		<key>onesuperior</key>
		<integer>60</integer>
		<key>question</key>
		<integer>60</integer>
		<key>threesuperior</key>
		<integer>60</integer>
		<key>twosuperior</key>
		<integer>60</integer>
	</dict>
	<key>X</key>
	<dict>
		<key>o</key>
		<integer>-60</integer>
		<key>s</key>
		<integer>-60</integer>
		<key>v</key>
		<integer>-60</integer>
		<key>w</key>
		<integer>-60</integer>
		<key>y</key>
		<integer>-60</integer>
	</dict>
	<key>Y</key>
	<dict>
		<key>a</key>
		<integer>-60</integer>
		<key>abreve</key>
		<integer>0</integer>
		<key>acircumflex</key>
		<integer>0</integer>
		<key>adieresis</key>
		<integer>0</integer>
		<key>agrave</key>
		<integer>0</integer>
		<key>amacron</key>
		<integer>0</integer>
		<key>aring</key>
		<integer>0</integer>
		<key>atilde</key>
		<integer>0</integer>
		<key>comma</key>
		<integer>-60</integer>
		<key>ibreve</key>
		<integer>60</integer>
		<key>icircumflex</key>
		<integer>60</integer>
		<key>idieresis</key>
		<integer>60</integer>
		<key>igrave</key>
		<integer>60</integer>
		<key>imacron</key>
		<integer>60</integer>
		<key>itilde</key>
		<integer>60</integer>
		<key>jcircumflex</key>
		<integer>60</integer>
		<key>o</key>
		<integer>-60</integer>
		<key>onesuperior</key>
		<integer>60</integer>
		<key>question</key>
		<integer>60</integer>
		<key>s</key>
		<integer>-60</integer>
		<key>threesuperior</key>
		<integer>60</integer>
		<key>twosuperior</key>
		<integer>60</integer>
	</dict>
	<key>comma</key>
	<dict>
		<key>onesuperior</key>
		<integer>-60</integer>
		<key>quotesingle</key>
		<integer>-120</integer>
		<key>threesuperior</key>
		<integer>-60</integer>
		<key>twosuperior</key>
		<integer>-60</integer>
		<key>uni2070</key>
		<integer>-60</integer>
		<key>uni2074</key>
		<integer>-120</integer>
	</dict>
	<key>f</key>
	<dict>
		<key>asterisk</key>
		<integer>120</integer>
		<key>exclam</key>
		<integer>60</integer>
		<key>onesuperior</key>
		<integer>120</integer>
		<key>question</key>
		<integer>120</integer>
		<key>quotesingle</key>
		<integer>120</integer>
		<key>threesuperior</key>
		<integer>120</integer>
		<key>twosuperior</key>
		<integer>120</integer>
		<key>uni2070</key>
		<integer>60</integer>
		<key>uni2074</key>
		<integer>60</integer>
	</dict>
	<key>f.alt</key>
	<dict>
		<key>asterisk</key>
		<integer>60</integer>
		<key>onesuperior</key>
		<integer>60</integer>
		<key>quotesingle</key>
		<integer>60</integer>
		<key>threesuperior</key>
		<integer>60</integer>
		<key>twosuperior</key>
		<integer>60</integer>
	</dict>
	<key>period</key>
	<dict>
		<key>onesuperior</key>
		<integer>-60</integer>
		<key>quotedblleft</key>
		<integer>-120</integer>
		<key>quotedblright</key>
		<integer>-120</integer>
		<key>quoteleft</key>
		<integer>-120</integer>
		<key>quoteright</key>
		<integer>-120</integer>
		<key>quotesingle</key>
		<integer>-120</integer>
		<key>threesuperior</key>
		<integer>-60</integer>
		<key>twosuperior</key>
		<integer>-60</integer>
		<key>uni2070</key>
		<integer>-60</integer>
		<key>uni2074</key>
		<integer>-120</integer>
	</dict>
	<key>questiondown.case</key>
	<dict>
		<key>A</key>
		<integer>60</integer>
	</dict>
</dict>
</plist>
```



Decsribing of A glyph from *Slabo 27px b002.ufo*

In [0]:
<?xml version="1.0" encoding="UTF-8"?>
<glyph name="A" format="1">
  <advance width="480"/>
  <unicode hex="0041"/>
  <outline>
    <contour>
      <point x="160" y="50" type="line"/>
      <point x="160" y="0" type="line"/>
      <point x="0" y="0" type="line"/>
      <point x="0" y="50" type="line"/>
      <point x="50" y="50" type="line"/>
      <point x="211" y="545" type="line"/>
      <point x="269" y="545" type="line"/>
      <point x="433" y="50" type="line"/>
      <point x="480" y="50" type="line"/>
      <point x="480" y="0" type="line"/>
      <point x="320" y="0" type="line"/>
      <point x="320" y="50" type="line"/>
      <point x="370" y="50" type="line"/>
      <point x="321" y="200" type="line"/>
      <point x="154" y="200" type="line"/>
      <point x="107" y="50" type="line"/>
    </contour>
    <contour>
      <point x="235" y="460" type="line"/>
      <point x="169" y="250" type="line"/>
      <point x="305" y="250" type="line"/>
      <point x="237" y="460" type="line"/>
    </contour>
  </outline>
</glyph>

Kerning from *Slabo 27px b002.ufo*

In [0]:
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>A</key>
	<dict>
		<key>O</key>
		<integer>-30</integer>
		<key>T</key>
		<integer>-30</integer>
		<key>U</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-60</integer>
		<key>W</key>
		<integer>-60</integer>
		<key>Y</key>
		<integer>-60</integer>
		<key>hyphen.case</key>
		<integer>-30</integer>
		<key>quotesingle</key>
		<integer>-60</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>B</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>C</key>
	<dict>
		<key>quotesingle</key>
		<integer>30</integer>
	</dict>
	<key>F</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>a</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
		<key>ibreve</key>
		<integer>30</integer>
		<key>icircumflex</key>
		<integer>30</integer>
		<key>idieresis</key>
		<integer>30</integer>
		<key>igrave</key>
		<integer>30</integer>
		<key>imacron</key>
		<integer>30</integer>
		<key>itilde</key>
		<integer>30</integer>
		<key>jcircumflex</key>
		<integer>30</integer>
		<key>o</key>
		<integer>-30</integer>
		<key>quotesingle</key>
		<integer>30</integer>
	</dict>
	<key>G</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>J</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>K</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>hyphen</key>
		<integer>-30</integer>
		<key>hyphen.case</key>
		<integer>-60</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>L</key>
	<dict>
		<key>T</key>
		<integer>-60</integer>
		<key>V</key>
		<integer>-60</integer>
		<key>W</key>
		<integer>-60</integer>
		<key>Y</key>
		<integer>-60</integer>
		<key>hyphen.case</key>
		<integer>-30</integer>
		<key>quotesingle</key>
		<integer>-30</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>N</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>O</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>X</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
		<key>Z</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
	</dict>
	<key>P</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>a</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-90</integer>
		<key>o</key>
		<integer>-30</integer>
	</dict>
	<key>R</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>S</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>T</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>a</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
		<key>guillemotleft</key>
		<integer>-30</integer>
		<key>guillemotleft.case</key>
		<integer>-30</integer>
		<key>guillemotright</key>
		<integer>-30</integer>
		<key>guilsinglleft</key>
		<integer>-30</integer>
		<key>guilsinglleft.case</key>
		<integer>-30</integer>
		<key>guilsinglright</key>
		<integer>-30</integer>
		<key>hyphen</key>
		<integer>-30</integer>
		<key>ibreve</key>
		<integer>30</integer>
		<key>icircumflex</key>
		<integer>30</integer>
		<key>idieresis</key>
		<integer>30</integer>
		<key>imacron</key>
		<integer>30</integer>
		<key>itilde</key>
		<integer>30</integer>
		<key>jcircumflex</key>
		<integer>30</integer>
		<key>o</key>
		<integer>-30</integer>
		<key>quotesingle</key>
		<integer>30</integer>
	</dict>
	<key>U</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
	</dict>
	<key>V</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>O</key>
		<integer>-30</integer>
		<key>a</key>
		<integer>-30</integer>
		<key>colon</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-90</integer>
		<key>guillemotleft</key>
		<integer>-30</integer>
		<key>guillemotleft.case</key>
		<integer>-30</integer>
		<key>guillemotright</key>
		<integer>-30</integer>
		<key>guilsinglleft</key>
		<integer>-30</integer>
		<key>guilsinglleft.case</key>
		<integer>-30</integer>
		<key>guilsinglright</key>
		<integer>-30</integer>
		<key>hyphen</key>
		<integer>-30</integer>
		<key>hyphen.case</key>
		<integer>-30</integer>
		<key>ibreve</key>
		<integer>30</integer>
		<key>icircumflex</key>
		<integer>30</integer>
		<key>idieresis</key>
		<integer>30</integer>
		<key>igrave</key>
		<integer>30</integer>
		<key>imacron</key>
		<integer>30</integer>
		<key>itilde</key>
		<integer>30</integer>
		<key>jcircumflex</key>
		<integer>30</integer>
		<key>m</key>
		<integer>-30</integer>
		<key>o</key>
		<integer>-30</integer>
		<key>s</key>
		<integer>-30</integer>
		<key>semicolon</key>
		<integer>-30</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>X</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>O</key>
		<integer>-30</integer>
		<key>guillemotleft.case</key>
		<integer>-30</integer>
		<key>guilsinglleft.case</key>
		<integer>-30</integer>
		<key>hyphen.case</key>
		<integer>-30</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>Y</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>O</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
		<key>a</key>
		<integer>-60</integer>
		<key>colon</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-90</integer>
		<key>guillemotleft</key>
		<integer>-60</integer>
		<key>guillemotleft.case</key>
		<integer>-30</integer>
		<key>guillemotright</key>
		<integer>-30</integer>
		<key>guilsinglleft</key>
		<integer>-60</integer>
		<key>guilsinglleft.case</key>
		<integer>-30</integer>
		<key>guilsinglright</key>
		<integer>-30</integer>
		<key>hyphen</key>
		<integer>-60</integer>
		<key>hyphen.case</key>
		<integer>-30</integer>
		<key>ibreve</key>
		<integer>30</integer>
		<key>icircumflex</key>
		<integer>30</integer>
		<key>idieresis</key>
		<integer>30</integer>
		<key>igrave</key>
		<integer>30</integer>
		<key>imacron</key>
		<integer>30</integer>
		<key>itilde</key>
		<integer>30</integer>
		<key>jcircumflex</key>
		<integer>30</integer>
		<key>m</key>
		<integer>-50</integer>
		<key>o</key>
		<integer>-60</integer>
		<key>s</key>
		<integer>-60</integer>
		<key>scaron</key>
		<integer>-30</integer>
		<key>semicolon</key>
		<integer>-30</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>Z</key>
	<dict>
		<key>O</key>
		<integer>-30</integer>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>exclamdown</key>
	<dict>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>f</key>
	<dict>
		<key>braceright</key>
		<integer>60</integer>
		<key>bracketright</key>
		<integer>60</integer>
		<key>comma</key>
		<integer>-30</integer>
		<key>exclam</key>
		<integer>30</integer>
		<key>ibreve</key>
		<integer>30</integer>
		<key>icircumflex</key>
		<integer>30</integer>
		<key>idieresis</key>
		<integer>30</integer>
		<key>igrave</key>
		<integer>30</integer>
		<key>imacron</key>
		<integer>30</integer>
		<key>itilde</key>
		<integer>30</integer>
		<key>jcircumflex</key>
		<integer>30</integer>
		<key>onesuperior</key>
		<integer>60</integer>
		<key>parenright</key>
		<integer>60</integer>
		<key>question</key>
		<integer>30</integer>
		<key>quotesingle</key>
		<integer>60</integer>
		<key>rcaron</key>
		<integer>30</integer>
		<key>scaron</key>
		<integer>30</integer>
		<key>threesuperior</key>
		<integer>60</integer>
		<key>twosuperior</key>
		<integer>60</integer>
		<key>uni2074</key>
		<integer>30</integer>
	</dict>
	<key>five</key>
	<dict>
		<key>seven</key>
		<integer>-30</integer>
	</dict>
	<key>four</key>
	<dict>
		<key>seven</key>
		<integer>-30</integer>
	</dict>
	<key>germandbls</key>
	<dict>
		<key>v</key>
		<integer>-30</integer>
	</dict>
	<key>guillemotleft</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>guillemotright</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-60</integer>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>guillemotright.case</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>X</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
	</dict>
	<key>guilsinglleft</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>guilsinglright</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-60</integer>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>guilsinglright.case</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>X</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-60</integer>
	</dict>
	<key>hyphen</key>
	<dict>
		<key>T</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-60</integer>
	</dict>
	<key>hyphen.case</key>
	<dict>
		<key>A</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>X</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>o</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>period</key>
	<dict>
		<key>guillemotleft</key>
		<integer>-30</integer>
		<key>guillemotleft.case</key>
		<integer>-30</integer>
		<key>guilsinglleft</key>
		<integer>-30</integer>
		<key>guilsinglleft.case</key>
		<integer>-30</integer>
		<key>quotesingle</key>
		<integer>-60</integer>
	</dict>
	<key>questiondown</key>
	<dict>
		<key>V</key>
		<integer>-30</integer>
		<key>W</key>
		<integer>-30</integer>
		<key>Y</key>
		<integer>-30</integer>
	</dict>
	<key>quotedblleft</key>
	<dict>
		<key>a</key>
		<integer>0</integer>
	</dict>
	<key>quoteleft</key>
	<dict>
		<key>a</key>
		<integer>0</integer>
	</dict>
	<key>quotesinglbase</key>
	<dict>
		<key>O</key>
		<integer>-30</integer>
		<key>T</key>
		<integer>-30</integer>
		<key>U</key>
		<integer>-30</integer>
		<key>V</key>
		<integer>-60</integer>
		<key>W</key>
		<integer>-60</integer>
		<key>Y</key>
		<integer>-60</integer>
		<key>v</key>
		<integer>-30</integer>
		<key>x</key>
		<integer>30</integer>
	</dict>
	<key>quotesingle</key>
	<dict>
		<key>A</key>
		<integer>-60</integer>
		<key>a</key>
		<integer>-30</integer>
		<key>comma</key>
		<integer>-90</integer>
		<key>o</key>
		<integer>-30</integer>
		<key>s</key>
		<integer>0</integer>
		<key>v</key>
		<integer>30</integer>
		<key>x</key>
		<integer>30</integer>
	</dict>
	<key>r</key>
	<dict>
		<key>comma</key>
		<integer>-30</integer>
	</dict>
	<key>seven</key>
	<dict>
		<key>four</key>
		<integer>-60</integer>
		<key>three</key>
		<integer>30</integer>
	</dict>
	<key>six</key>
	<dict>
		<key>seven</key>
		<integer>-30</integer>
	</dict>
	<key>v</key>
	<dict>
		<key>comma</key>
		<integer>-60</integer>
		<key>quotesingle</key>
		<integer>30</integer>
	</dict>
	<key>x</key>
	<dict>
		<key>quotesingle</key>
		<integer>30</integer>
	</dict>
</dict>
</plist>

https://docs.python.org/2/library/xml.etree.elementtree.html